## <center> Sentiment Analysis Of Movie Reviews </center>

In [1]:
import pandas as pd
import os

In [2]:
# Load data
rating_df = pd.read_csv('data/rating_auto_label_sentiment_two_classes.csv')

# drop unused columns
rating_df = rating_df [['review_text','sentiment']]
rating_df.head(2)
rating_df.shape

(10468, 2)

In [3]:
# Drop rows with NaN values in any column
rating_df = rating_df.dropna()
rating_df.shape

(10462, 2)

In [4]:
import re
import string

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from collections import Counter

# Text Pre-processing
lemmatizer=WordNetLemmatizer()

def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'): #ADJECTIVE
        return wordnet.ADJ
    elif nltk_tag.startswith('V'): #VERN
        return wordnet.VERB
    elif nltk_tag.startswith('N'): #NOUN        
        return wordnet.NOUN
    elif nltk_tag.startswith('R'): #ADVERB
        return wordnet.ADV
    else:          
        return None

def lemmatize_sentence(sentence):
    # Tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    # Tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged) 
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            # If no tag was found, then use the word as is
            lemmatized_sentence.append(word)
        else:        
            # Else use the tag to lemmatize the word
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)

def preprocess_text(df):
    df['review_text'] = df['review_text'].astype(str).fillna('')

    # remove white space
    df['review_text'] = df['review_text'].str.strip().str.replace(r'\s+', ' ', regex=True)

    # update to lower case
    df['review_text'] = df['review_text'].str.lower()

    # remove punctuations
    df['review_text'] = df['review_text'].str.replace(r'[{}]'.format(re.escape(string.punctuation)), '', regex=True)

    # remove special characters
    df['review_text'] = df['review_text'].str.replace(r'[^\x00-\x7F]+', '', regex=True)

    # remove digits
    df['review_text'] = df['review_text'].str.replace(r'\d+', '', regex=True)

    # remove non ascii
    df['review_text'] = df['review_text'].str.replace(r'[^\x00-\x7F]+', '', regex=True)
    
    return df

def remove_stopwords(df):
    stop_words = stopwords.words('english') + ['br']
    stopwords_dict = Counter(stop_words)
    df['review_text'] = df['review_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords_dict]))

    return df

def lemmatize(df):
    df['review_text'] = df['review_text'].apply(lambda x: lemmatize_sentence(x))

    return df

df_result = pd.DataFrame(columns=['model', 'task_no', 'vectorizer', 'ngram', 'max_iter', 'C', 'gamma', 'tree_param', 'n_estimator', 'lrate', 'test_accuracy', 'wall_time','run_time'])
model_no = 1
filename="output/result_SVM_RBF.csv"

In [5]:
# Text Preprocessing
rating_df = preprocess_text(rating_df)
rating_df = remove_stopwords(rating_df)
rating_df = lemmatize(rating_df)

rating_df.head()

,review_text,sentiment
0,let leave door love beetlejuice edward scissor...,NEGATIVE
1,fast paced action thriller delivers begin end ...,POSITIVE
2,excellent movie great cast see movie saw one r...,POSITIVE
3,write three highpraise review try think bad mo...,NEGATIVE
4,well make movie quality write act cinematograp...,POSITIVE


In [6]:
from sklearn.model_selection import train_test_split

# X_train,X_test,y_train,y_test = train_test_split(rating_df.review_text,rating_df.sentiment,test_size = 0.2, random_state=42)

# 80% training, 20% temporary
X_train, X_temp, y_train, y_temp = train_test_split(rating_df.review_text, rating_df.sentiment, test_size=0.2, random_state=42)

# 10% validation, 10% test
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(8369,)
(1046,)
(1047,)


In [7]:
# SVM Model

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import time 
from datetime import datetime

n_vects = ['cbow','tfidf']
n_grams = [1,2,3]
n_iters = [100000]
Cs = [0.01, 1, 100]
gammas = ['scale', 'auto']
# n_stop_lemmatize = [False, True]

best_model = ''
best_accuracy = 0
best_y_test_pred = None

for n_gram in n_grams:
    for n_vect in n_vects:
        if n_vect=='cbow':
            # Use all features, remove stopwords, apply unigram, bigram, trigram
            vect = CountVectorizer(max_features=None, ngram_range=(1,n_gram), stop_words='english', lowercase=True, strip_accents='ascii')
        else:
            vect = TfidfVectorizer(max_features=None, ngram_range=(1,n_gram), stop_words='english', lowercase=True, strip_accents='ascii')

        # Fit on training data and transform the training data to vector (document-term matrix)
        X_train_dtm = vect.fit_transform(X_train)
        # display(X_train_dtm)

        X_val_dtm = vect.transform(X_val)
        # display(X_val_dtm)

        X_test_dtm = vect.transform(X_test)
        # display(X_test_dtm)

        for n_iter in n_iters:
            for C in Cs:
                for gamma in gammas:
                    # Initialize the LogisticRegression classifier
                    svm_rbf = SVC (max_iter=n_iter, kernel='rbf', C=C, gamma=gamma, class_weight='balanced', random_state=42)

                    # Train the classifier on the training data & capture wall time
                    start_time = time.time()
                    %time svm_rbf.fit(X_train_dtm, y_train)
                    end_time = time.time()
                    wall_time = end_time - start_time

                    # Predict and evaluate the classifier
                    y_val_pred = svm_rbf.predict(X_val_dtm)
                    val_accuracy = accuracy_score(y_val, y_val_pred)

                    # Final evaluation on test set
                    y_test_pred = svm_rbf.predict(X_test_dtm)
                    test_accuracy = accuracy_score(y_test, y_test_pred)

                    # Print result
                    task_no = str(model_no)
                    model = 'RbfSVM'
                    print(f"{model} - {task_no}, text_preprocess: {True}, vectorizer: {n_vect}, ngram: {n_gram}, max_iter: {n_iter}, C:{C}, gamma:{gamma}")
                    print(f"Test Accuracy: {test_accuracy}\n")
                    model_no +=1

                    # Record result to dataframe, to be exported to csv
                    # columns=['model', 'task_no', 'vectorizer', 'ngram', 'max_iter', 'C', 'gamma', 'tree_param', 'n_estimator', 'lrate', 'test_accuracy', 'wall_time','run_time']
                    new_row = [model, task_no, n_vect, n_gram, n_iter, C, gamma, '', 0, 0, test_accuracy, wall_time, datetime.now().strftime("%Y-%m-%d %H:%M:%S")]
                    df_result.loc[len(df_result)] = new_row

                    new_row_df = pd.DataFrame([new_row], columns=df_result.columns)
                    new_row_df.to_csv(filename, index=False, mode='a', header=not os.path.exists(filename))

                    # Check for the best model
                    if test_accuracy > best_accuracy:
                        best_model = f"{model} - {task_no}, text_preprocess: {True}, vectorizer: {n_vect}, ngram: {n_gram}, max_iter: {n_iter}, C:{C}, gamma:{gamma}"
                        best_accuracy = test_accuracy
                        best_y_test_pred = y_test_pred

CPU times: total: 13.3 s
Wall time: 1min 2s
RbfSVM - 1, text_preprocess: True, vectorizer: cbow, ngram: 1, max_iter: 100000, C:0.01, gamma:scale
Test Accuracy: 0.37058261700095513

CPU times: total: 50.6 s
Wall time: 1min 21s
RbfSVM - 2, text_preprocess: True, vectorizer: cbow, ngram: 1, max_iter: 100000, C:0.01, gamma:auto
Test Accuracy: 0.37058261700095513

CPU times: total: 54.8 s
Wall time: 1min 16s
RbfSVM - 3, text_preprocess: True, vectorizer: cbow, ngram: 1, max_iter: 100000, C:1, gamma:scale
Test Accuracy: 0.7459407831900668

CPU times: total: 1min 5s
Wall time: 1min 27s
RbfSVM - 4, text_preprocess: True, vectorizer: cbow, ngram: 1, max_iter: 100000, C:1, gamma:auto
Test Accuracy: 0.6876790830945558

CPU times: total: 40.6 s
Wall time: 1min 20s
RbfSVM - 5, text_preprocess: True, vectorizer: cbow, ngram: 1, max_iter: 100000, C:100, gamma:scale
Test Accuracy: 0.7211079274116523

CPU times: total: 14.9 s
Wall time: 49.2 s
RbfSVM - 6, text_preprocess: True, vectorizer: cbow, ngram:

In [8]:
# Print the classification report of the best model
from sklearn.metrics import accuracy_score, classification_report

if best_y_test_pred is not None:
    print("The best model:", best_model)
    print("The best accuracy:", best_accuracy)

    Report=classification_report(y_test,best_y_test_pred)
    print("Classification Report of the Best Model:\n")
    print(Report)

The best model: RbfSVM - 21, text_preprocess: True, vectorizer: tfidf, ngram: 2, max_iter: 100000, C:1, gamma:scale
The best accuracy: 0.7631327602674307
Classification Report of the Best Model:

              precision    recall  f1-score   support

    NEGATIVE       0.73      0.58      0.64       388
    POSITIVE       0.78      0.87      0.82       659

    accuracy                           0.76      1047
   macro avg       0.75      0.72      0.73      1047
weighted avg       0.76      0.76      0.76      1047



In [9]:
# Inference on new data
# new_reviews = ['A worthy contender for the Animated film of 2024', 'No plot at all. But if you are looking for a good laugh. You will not find that either.']
new_reviews = [
    "I absolutely love this movie! It was amazing.",
    "This movie was terrible, I hated every second of it.", 
    "while this movie is not intended for everyone, it is good for someone has no brain", 
    "let's watch it only when it is free to watch, i will not pay for it",
    'A worthy contender for the Animated film of 2024', 
    'No plot at all. But if you are looking for a good laugh. You will not find that either.'
]

new_reviews_dtm = vect.transform(new_reviews)
new_predictions = svm_rbf.predict(new_reviews_dtm)

print("New Predictions:", new_predictions)

New Predictions: ['POSITIVE' 'POSITIVE' 'POSITIVE' 'POSITIVE' 'POSITIVE' 'POSITIVE']
